In [1]:
from Bio import Entrez
from datetime import datetime
import json
import xmltodict
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [41]:
def clean_dict(dictionary):
    dt_new = {}
    for key in dictionary:
        if isinstance(dictionary[key], dict):
            dt_new.update(clean_dict(dictionary[key]))
        else:
            dt_new[key] = dictionary[key]
    return dt_new

def dict_to_df(dictionary, df: pd.DataFrame):
    dt_new = clean_dict(dictionary)
    df = pd.concat([df, pd.DataFrame([dt_new], index=[0])], ignore_index=True)
    return df

In [44]:

# Configura tu dirección de correo electrónico (es necesario para usar la API)
Entrez.email = "josemanuelgonzalezfornell@gmail.com"

# Obtén la fecha actual y calcula la fecha de hace un año
fecha_actual = datetime.now()
fecha_hace_un_anio = fecha_actual.replace(year=fecha_actual.year - 1)

# Convierte las fechas al formato necesario para la búsqueda en PubMed
fecha_actual_str = fecha_actual.strftime("%Y/%m/%d")
fecha_hace_un_anio_str = fecha_hace_un_anio.strftime("%Y/%m/%d")

max_results=10000
retmax=100
all_results = []

# Realiza la búsqueda en PubMed
for retstart in range(0, max_results, retmax):
    handle = Entrez.esearch(db="pubmed", term=f'"{fecha_hace_un_anio_str}"[Date - Publication] : "{fecha_actual_str}"[Date - Publication]', retmax=retmax, retstart=retstart)
    record = Entrez.read(handle)
    handle.close()
    all_results.extend(record["IdList"])


handle = Entrez.efetch(db="pubmed", id=all_results, retmode="xml")
record = handle.read()
handle.close()

df = pd.DataFrame()

# Descarga la información de cada paper
for id_paper in all_results:
    handle = Entrez.efetch(db="pubmed", id=id_paper)
    record = handle.read()
    handle.close()

    # Convierte XML a JSON usando xmltodict
    json_data = xmltodict.parse(record)
    
    df = dict_to_df(json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"], df)

df





,@Status,@Owner,@IndexingMethod,@Version,#text,Year,Month,Day,@PubModel,@IssnType,...,mml:mi,mml:msqrt,CollectiveName,DataBank,u,Note,mml:mn,mml:mtext,MedlineDate,mml:annotation
0,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,24,Print-Electronic,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Publisher,NOTNLM,Automated,1,Journal Article,2023,Oct,24,Print,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Publisher,NLM,Automated,1,Journal Article,2023,10,24,Print-Electronic,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,24,Print-Electronic,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Publisher,NOTNLM,Automated,1,Journal Article,2023,Oct,24,Print,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,18,Print-Electronic,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9995,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,18,Print-Electronic,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,18,Print-Electronic,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,18,Print-Electronic,Electronic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.to_csv("pubmed.csv")

In [8]:
df = pd.read_csv("/Users/administrador/Documents/proyectos/ncbi/src/data/raw/pubmed.csv", index_col=0)

/var/folders/r1/hd_91nzn1yb8kh1vq1c7gt2w0000gn/T/ipykernel_1873/2584090995.py:1: DtypeWarning: Columns (31,60,65,66,67,69,70,73,75,77,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/administrador/Documents/proyectos/ncbi/src/data/raw/pubmed.csv", index_col=0)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9999 entries, 0 to 9998
Data columns (total 81 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   @Status               9999 non-null   object 
 1   @Owner                9999 non-null   object 
 2   @IndexingMethod       9999 non-null   object 
 3   @Version              9999 non-null   int64  
 4   #text                 9999 non-null   object 
 5   Year                  9999 non-null   int64  
 6   Month                 9999 non-null   object 
 7   Day                   9999 non-null   int64  
 8   @PubModel             9999 non-null   object 
 9   @IssnType             9570 non-null   object 
 10  @CitedMedium          9999 non-null   object 
 11  Title                 9999 non-null   object 
 12  ISOAbbreviation       9999 non-null   object 
 13  ArticleTitle          9384 non-null   object 
 14  @EIdType              4559 non-null   object 
 15  @ValidYN              4809

In [7]:
pd.set_option("display.max_columns", None)
df.sample(10)

,Unnamed: 0,@Status,@Owner,@IndexingMethod,@Version,#text,Year,Month,Day,@PubModel,@IssnType,@CitedMedium,Title,ISOAbbreviation,ArticleTitle,@EIdType,@ValidYN,AbstractText,CopyrightInformation,@CompleteYN,Author,Language,Grant,@UI,@DateType,Country,MedlineTA,NlmUniqueID,ISSNLinking,CitationSubset,Keyword,Volume,Issue,i,StartPage,EndPage,MedlinePgn,PublicationType,CoiStatement,Acronym,Agency,ELocationID,@RefType,RefSource,sup,sub,LastName,ForeName,Initials,Affiliation,GrantID,@Source,AffiliationInfo,b,@Type,@Language,VernacularTitle,Investigator,Season,@xmlns:mml,mml:mrow,CommentsCorrections,DataBankName,AccessionNumber,@VersionID,mml:math,Suffix,Identifier,@MajorTopicYN,OtherAbstract,mml:msub,mml:mo,mml:mi,mml:msqrt,CollectiveName,DataBank,u,Note,mml:mn,mml:mtext,MedlineDate,mml:annotation
6282,6282,Publisher,NLM,Automated,1,Journal Article,2023,10,23,Electronic,Electronic,Internet,The American journal of case reports,Am J Case Rep,A Case Report of Oral Bisphosphonate Treatment...,doi,Y,BACKGROUND Bisphosphonates inhibit bone resorp...,NaN,Y,"[{'@ValidYN': 'Y', 'LastName': 'Fernandes', 'F...",eng,NaN,D016428,Electronic,United States,Am J Case Rep,101489566,1941-5923,IM,NaN,24,NaN,NaN,e941144,NaN,e941144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8869,8869,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,21,Print-Electronic,Electronic,Internet,Journal of genetic counseling,J Genet Couns,"""Out of the blue"": A qualitative study explori...",doi,Y,"In the genomic era, the availability of gene p...",© 2023 The Authors. Journal of Genetic Counsel...,Y,"[{'@ValidYN': 'Y', 'LastName': 'Morrow', 'Fore...",eng,"[{'GrantID': 'NT-15-016', 'Agency': 'National ...",D016428,Electronic,United States,J Genet Couns,9206865,1059-7700,IM,"[{'@MajorTopicYN': 'N', '#text': 'adaptation'}...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3605,3605,Publisher,NOTNLM,Automated,1,Journal Article,2023,10,20,Print-Electronic,Electronic,Internet,Current Alzheimer research,Curr Alzheimer Res,Development of Computational Correlations amon...,doi,Y,"[{'@Label': 'BACKGROUND', '@NlmCategory': 'BAC...",Copyright© Bentham Science Publishers; For any...,Y,"[{'@ValidYN': 'Y', 'LastName': 'Roy', 'ForeNam...",eng,NaN,D016428,Electronic,United Arab Emirates,Curr Alzheimer Res,101208441,1567-2050,IM,"[{'@MajorTopicYN': 'N', '#text': 'ACHE pathway...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6229,6229,Publisher,NLM,Automated,1,Journal Article,2023,10,22,Print-Electronic,Electronic,Internet,American journal of human biology : the offici...,Am J Hum Biol,Risk factors associated with weak and asymmetr...,doi,Y,"[{'@Label': 'OBJECTIVES', '@NlmCategory': 'OBJ...",© 2023 Wiley Periodicals LLC.,Y,"[{'@ValidYN': 'Y', 'LastName': 'Weng', 'ForeNa...",eng,"[{'GrantID': 'S-2022-000166', 'Agency': 'The 2...",D016428,Electronic,United States,Am J Hum Biol,8915029,1042-0533,IM,NaN,NaN,NaN,NaN,e24007,NaN,e24007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5113,5113,Publisher,NOTNLM,Automated,1,Case Reports,2023,09,21,Electronic-eCollection,Print,Print,Cureus,Cureus,Idiopathic Sclerosing Encapsulating Peritoniti...,NaN,NaN,"During our practice as clinical surgeons, we h...","Copyright © 2023, Skarmoutsou et al.",Y,"[{'@ValidYN': 'Y', 'LastName': 'Skarmoutsou', ...",eng,NaN,D002363,Electronic,United States,Cureus,101596737,2168-8184,NaN,"[{'@MajorTopicYN': 'N', '#text': 'abdominal co...",15,9,NaN,e4567

In [20]:
a = clean_dict(json_data["PubmedArticleSet"]["PubmedArticle"])

In [21]:
a["@Status"]

'Publisher'

In [22]:
pd.DataFrame([a], index=[0])

,@Status,@Owner,@IndexingMethod,@Version,#text,Year,Month,Day,@PubModel,@IssnType,@CitedMedium,Volume,Issue,Title,ISOAbbreviation,ArticleTitle,StartPage,MedlinePgn,@EIdType,@ValidYN,AbstractText,CopyrightInformation,@CompleteYN,Author,Language,Grant,@UI,@DateType,Country,MedlineTA,NlmUniqueID,ISSNLinking,CitationSubset,PubMedPubDate,PublicationStatus,ArticleId,Reference
0,Publisher,NLM,Automated,1,Journal Article,2023,10,21,Electronic,Electronic,Internet,13,1,Scientific reports,Sci Rep,Testosterone promotes dominance behaviors in t...,18029,18029,doi,Y,Although testosterone is generally considered ...,© 2023. Springer Nature Limited.,Y,"[{'@ValidYN': 'Y', 'LastName': 'Inoue', 'ForeN...",eng,"[{'GrantID': 'JP15K13115', 'Agency': 'Japan So...",D016428,Electronic,England,Sci Rep,101563288,2045-2322,IM,"[{'@PubStatus': 'received', 'Year': '2023', 'M...",epublish,"[{'@IdType': 'pubmed', '#text': '37865708'}, {...","[{'Citation': 'Mazur, A. & Booth, A. Testoster..."


In [23]:
df = pd.DataFrame()

dict_to_df(json_data["PubmedArticleSet"]["PubmedArticle"], df)

,@Status,@Owner,@IndexingMethod,@Version,#text,Year,Month,Day,@PubModel,@IssnType,@CitedMedium,Volume,Issue,Title,ISOAbbreviation,ArticleTitle,StartPage,MedlinePgn,@EIdType,@ValidYN,AbstractText,CopyrightInformation,@CompleteYN,Author,Language,Grant,@UI,@DateType,Country,MedlineTA,NlmUniqueID,ISSNLinking,CitationSubset,PubMedPubDate,PublicationStatus,ArticleId,Reference
0,Publisher,NLM,Automated,1,Journal Article,2023,10,21,Electronic,Electronic,Internet,13,1,Scientific reports,Sci Rep,Testosterone promotes dominance behaviors in t...,18029,18029,doi,Y,Although testosterone is generally considered ...,© 2023. Springer Nature Limited.,Y,"[{'@ValidYN': 'Y', 'LastName': 'Inoue', 'ForeN...",eng,"[{'GrantID': 'JP15K13115', 'Agency': 'Japan So...",D016428,Electronic,England,Sci Rep,101563288,2045-2322,IM,"[{'@PubStatus': 'received', 'Year': '2023', 'M...",epublish,"[{'@IdType': 'pubmed', '#text': '37865708'}, {...","[{'Citation': 'Mazur, A. & Booth, A. Testoster..."


In [26]:
clean_dict(json_data["PubmedArticleSet"]["PubmedArticle"])

{'@Status': 'Publisher',
 '@Owner': 'NLM',
 '@IndexingMethod': 'Automated',
 '@Version': '1',
 '#text': 'Journal Article',
 'Year': '2023',
 'Month': '10',
 'Day': '21',
 '@PubModel': 'Electronic',
 '@IssnType': 'Electronic',
 '@CitedMedium': 'Internet',
 'Volume': '13',
 'Issue': '1',
 'Title': 'Scientific reports',
 'ISOAbbreviation': 'Sci Rep',
 'ArticleTitle': "Testosterone promotes dominance behaviors in the Ultimatum Game after players' status increases.",
 'StartPage': '18029',
 'MedlinePgn': '18029',
 '@EIdType': 'doi',
 '@ValidYN': 'Y',
 'AbstractText': "Although testosterone is generally considered to promote dominance behaviors, in humans it fosters behaviors appropriate to achieving and maintaining social status, contingent upon the situation. Recent cross-sectional studies, such as Inoue et al. (Sci Rep 7:5335, 2017), have shown that dominance behaviors induced by testosterone are modulated by high status. Yet, it remains ambiguous whether a rise in social status within re